In [10]:
import os

file_path = r"D:\physic\data\val\val_20_30_small.h5"
if os.path.exists(file_path):
    print(f"File exists: {file_path}")
else:
    print(f"File does not exist: {file_path}")


File exists: D:\physic\data\val\val_20_30_small.h5


In [11]:
import random
import h5py
from JetDataset import JetDataset
import torch
from torch.utils.data import Dataset, DataLoader
tag = 'train'
# train_dataset = JetDataset(f"/mnt/d/physic/data/train_20_30_new.h5", config=None)
train_dataset = JetDataset(r"D:\physic\data\val\val_20_30_small.h5", config=None)





Initializing JetDataset with file: D:\physic\data\val\val_20_30_small.h5
Loading features and subjets from HDF5 file
Filtered to 6452 good jets
Final dataset size: 6452 jets
__getitem__ returns (x, particle_features, subjets, indices, subjet_mask, particle_mask)


In [12]:
train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)
train_iter = iter(train_loader)
for i in range(len(train_dataset) // 1000 + 1):
    x, particle_features, subjets, particle_indices, subjet_mask, particle_mask = next(train_iter)

In [13]:
x.shape

torch.Size([452, 20, 30, 4])

In [14]:
from tqdm import tqdm
import h5py
import torch
import numpy as np
from torch.utils.data import DataLoader

# Initialize DataLoader
batch_size = 1000
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create a new HDF5 file
# out_path = f"D:\physic\data\processed_{tag}_20_30_torch.h5"
out_path = r"D:\physic\data\processed_val_20_30_torch.h5"
with h5py.File(out_path, 'w') as hf:
    # Get the first batch to determine sizes
    first_batch = next(iter(train_loader))
    num_jets = len(train_dataset)
    num_subjets = first_batch[0].shape[1]
    num_ptcls_per_subjet = first_batch[0].shape[2]
    num_ptcl_ftrs = first_batch[0].shape[3]
    num_ptcls_per_jet = first_batch[1].shape[1]
    num_subjet_ftrs = first_batch[2].shape[2]

    # Initialize datasets with extendable dimensions in the first dimension
    hf.create_dataset('x', shape=(0, num_subjets, num_ptcls_per_subjet, num_ptcl_ftrs), maxshape=(None, num_subjets, num_ptcls_per_subjet, num_ptcl_ftrs), dtype='float32')
    hf.create_dataset('particle_features', shape=(0, num_ptcls_per_jet, num_ptcl_ftrs), maxshape=(None, num_ptcls_per_jet, num_ptcl_ftrs), dtype='float64')
    hf.create_dataset('subjets', shape=(0, num_subjets, num_subjet_ftrs), maxshape=(None, num_subjets, num_subjet_ftrs), dtype='float64')
    hf.create_dataset('particle_indices', shape=(0, num_subjets, num_ptcls_per_subjet), maxshape=(None, num_subjets, num_ptcls_per_subjet), dtype='int32')
    hf.create_dataset('subjet_mask', shape=(0, num_subjets), maxshape=(None, num_subjets), dtype='bool')
    hf.create_dataset('particle_mask', shape=(0, num_subjets, num_ptcls_per_subjet), maxshape=(None, num_subjets, num_ptcls_per_subjet), dtype='bool')

    # Write batches to HDF5
    num_batches_processed = 0
    for data in tqdm(train_loader):
        x, particle_features, subjets, particle_indices, subjet_mask, particle_mask = [d.detach().cpu() for d in data]

        # convert to appropriate data types

        # Number of new data points
        num_new = x.shape[0]
        # particle_indices = particle_indices.astype(np.int32)
        # subjet_mask = subjet_mask.astype(np.int32)
        # particle_mask = particle_mask.astype(np.int32)

        particle_indices = particle_indices.to(torch.int32)
        subjet_mask = subjet_mask.to(torch.int32)
        particle_mask = particle_mask.to(torch.int32)

        # Resize datasets to accommodate new data
        hf['x'].resize(num_batches_processed * batch_size + num_new, axis=0)
        hf['particle_features'].resize(num_batches_processed * batch_size + num_new, axis=0)
        hf['subjets'].resize(num_batches_processed * batch_size + num_new, axis=0)
        hf['particle_indices'].resize(num_batches_processed * batch_size + num_new, axis=0)
        hf['subjet_mask'].resize(num_batches_processed * batch_size + num_new, axis=0)
        hf['particle_mask'].resize(num_batches_processed * batch_size + num_new, axis=0)

        # Append new data
        hf['x'][num_batches_processed * batch_size:num_batches_processed * batch_size + num_new] = x
        hf['particle_features'][num_batches_processed * batch_size:num_batches_processed * batch_size + num_new] = particle_features
        hf['subjets'][num_batches_processed * batch_size:num_batches_processed * batch_size + num_new] = subjets
        hf['particle_indices'][num_batches_processed * batch_size:num_batches_processed * batch_size + num_new] = particle_indices
        hf['subjet_mask'][num_batches_processed * batch_size:num_batches_processed * batch_size + num_new] = subjet_mask
        hf['particle_mask'][num_batches_processed * batch_size:num_batches_processed * batch_size + num_new] = particle_mask

        num_batches_processed += 1


100%|██████████| 7/7 [00:02<00:00,  2.75it/s]


In [15]:
from torch.utils.data import Dataset, DataLoader
import torch
import h5py

class HDF5Dataset(Dataset):
    def __init__(self, file_path):
        with h5py.File(file_path, 'r') as self.file:
            self.x = self.file['x'][:] # Load entire dataset into memory
            self.particle_features = self.file['particle_features'][:]
            self.subjets = self.file['subjets'][:]
            self.particle_indices = self.file['particle_indices'][:]
            self.subjet_mask = self.file['subjet_mask'][:]
            self.particle_mask = self.file['particle_mask'][:]

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        # Directly return the data from memory
        return (self.x[idx], self.particle_features[idx], self.subjets[idx],
                self.particle_indices[idx], self.subjet_mask[idx], self.particle_mask[idx])

# Usage
tag = 'train'
# dataset_path = f"D:\physic\data\processed_{tag}_20_30_torch.h5"
dataset_path = r"D:\physic\data\processed_val_20_30_torch.h5"

dataset = HDF5Dataset(dataset_path)


In [16]:
train_loader = DataLoader(dataset, batch_size=1000, shuffle=True)
train_iter = iter(train_loader)
for i in range(len(dataset) // 1000 + 1):
    x, particle_features, subjets, particle_indices, subjet_mask, particle_mask = next(train_iter)

In [17]:
len(dataset)

6452